[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/swati-rajwal/BreastCancer_tweets_project/blob/main/E_Train_size_vs_performance.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -q transformers

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
# import emoji
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras


#metrics
from sklearn.metrics import accuracy_score, f1_score, make_scorer, fbeta_score
from sklearn.metrics import classification_report, confusion_matrix

#set seed for reproducibility
seed=42

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Name : Avinash Kumar Pandey
# Final Project Task 1
# BMI NLP Course

# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, make_scorer, log_loss
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import gensim
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-3-564121d0e846>:46: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading pack

True

<Figure size 640x480 with 0 Axes>

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/BioNLP Final project/Breast Cancer Data/Classification_ data/train.csv')
df_dev = pd.read_csv('/content/drive/MyDrive/BioNLP Final project/Breast Cancer Data/Classification_ data/dev_.csv')
df_test = pd.read_csv('/content/drive/MyDrive/BioNLP Final project/Breast Cancer Data/Classification_ data/test.csv')

In [ ]:
print(f"Train set shape: {df_train.shape}")
print(f"Dev set shape: {df_dev.shape}")
print(f"Test set shape: {df_test.shape}")

Train set shape: (3513, 2)
Dev set shape: (302, 2)
Test set shape: (1204, 2)


In [ ]:
df_train.columns

Index(['text', 'Class'], dtype='object')

In [ ]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
df_train.sample(3)

,text,Class
959,<user> i've enjoyed u all these years & will m...,1
2068,"<user> as a military wife, breast cancer survi...",0
1985,<user> wishing upon an <hashtag> alaskahappyho...,1


In [ ]:
# def preprocess(text):
#     text = text.lower()
#     tokens = nltk.word_tokenize(text)
#     filtered = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english') and word not in string.punctuation]
#     return ' '.join(filtered)


# df_train['before_preprocess_length'] = df_train['medical_abstract'].apply(lambda x: len(str(x)))
# print("############### Before preprocessing ##########")
# print(df_train['before_preprocess_length'].max())
# print(df_train['before_preprocess_length'].min())



  # Pre-load the stopwords
STOPWORDS = set(stopwords.words('english'))

def remove_emojis(text):
    # Regular expression to match emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def preprocess_text_optimized(text):
    text = remove_emojis(text)
    text = text.lower()
    # Remove <word> patterns
    text = re.sub(r'<[^>]+>', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace('@', '').replace('\n', '')
    tokens = word_tokenize(text)
    # Filter out stopwords
    # tokens = nltk.word_tokenize(text)
    filtered = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english') and word not in string.punctuation]
    return ' '.join(filtered)

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:

df_train['text']= df_train['text'].apply(preprocess_text_optimized)
df_dev['text'] = df_dev['text'].apply(preprocess_text_optimized)
df_test['text'] = df_test['text'].apply(preprocess_text_optimized)
# print("############### Before preprocessing ##########")
# print(df_train['after_preprocess_length'].max())
# print(df_train['after_preprocess_length'].min())

In [ ]:
df_train.sample(3)

,text,Class
1002,’ breastcancerawareness month tomorrow gmsa su...,0
1619,following breastcancer diagnosis want yearofgr...,1
3237,’ honor participate pink patch project serve c...,0


In [ ]:
df_train.iloc[1002]

text     ’ breastcancerawareness month tomorrow gmsa su...
Class                                                    0
Name: 1002, dtype: object

In [ ]:
# Run this after preprocessing the texts
token_lens = []

for txt in df_train['text'].values:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))

max_len=np.max(token_lens)

print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 114


In [ ]:
results_df = pd.DataFrame(columns=['Model name', 'Train set size', 'Accuracy', 'f1_micro', 'f1_macro', 'f2_micro', 'f2_macro', 'logloss'])

def create_model(bert_model, max_len):

      opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
      loss = tf.keras.losses.CategoricalCrossentropy()
      accuracy = tf.keras.metrics.CategoricalAccuracy()

      input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
      attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
      output = bert_model([input_ids,attention_masks])
      output = output[1]
      output = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(output)
      model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
      model.compile(opt, loss=loss, metrics=accuracy)
      return model

def tokenize_roberta(data, max_len):
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            truncation=True,  # Explicitly activate truncation
            padding='max_length',  # Update padding argument
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids), np.array(attention_masks)

def conf_matrix(y, y_pred, title):
      fig, ax =plt.subplots(figsize=(5,5))
      labels=[0, 1]
      ax=sns.heatmap(confusion_matrix(y, y_pred), annot=True, cmap="Blues", fmt='g', cbar=False, annot_kws={"size":25})
      plt.title(title, fontsize=20)
      ax.xaxis.set_ticklabels(labels, fontsize=17)
      ax.yaxis.set_ticklabels(labels, fontsize=17)
      ax.set_ylabel('Test', fontsize=20)
      ax.set_xlabel('Predicted', fontsize=20)
      plt.show()


for size in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
  print("*******************************************")
  print("Currently running for size: ",size)
  df = pd.read_csv('/content/drive/MyDrive/BioNLP Final project/Breast Cancer Data/Classification_ data/train.csv')
  df_test = pd.read_csv('/content/drive/MyDrive/BioNLP Final project/Breast Cancer Data/Classification_ data/dev_.csv')

  # df_train, temp = train_test_split(df, train_size= size, random_state= 20)

  df['text'] = df['text'].apply(preprocess_text_optimized)
  df['text'] = df['text'].apply(preprocess_text_optimized)

  X = df['text'].values
  y = df['Class'].values
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=size, stratify=y, random_state=seed)

  X_test = df_test['text'].values
  y_test = df_test['Class'].values

  y_train_le = y_train.copy()
  y_valid_le = y_valid.copy()
  y_test_le = y_test.copy()

  ohe = preprocessing.OneHotEncoder()
  y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
  y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()
  y_test = ohe.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

  clf = CountVectorizer()
  X_train_cv =  clf.fit_transform(X_train)
  X_test_cv = clf.transform(X_test)

  tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_cv)
  X_train_tf = tf_transformer.transform(X_train_cv)
  X_test_tf = tf_transformer.transform(X_test_cv)

  # NAIVE BAYES
  nb_clf = MultinomialNB()
  nb_clf.fit(X_train_tf, y_train_le)
  nb_pred = nb_clf.predict(X_test_tf)
  results_df.loc[len(results_df.index)] = ['Naive bayes', size, accuracy_score(y_test.argmax(1), nb_pred),
                                           f1_score(y_test.argmax(1), nb_pred, average='micro'),
                                           f1_score(y_test.argmax(1), nb_pred, average='macro'),
                                           fbeta_score(y_test.argmax(1), nb_pred, beta=2, average='micro'),
                                           fbeta_score(y_test.argmax(1), nb_pred, beta=2, average='macro'),
                                           log_loss(y_test, nb_clf.predict_proba(X_test_tf))]
  print("Naive Bayes")
  # DECISION TREE
  d_model = DecisionTreeClassifier(criterion='gini', max_depth=10)
  d_model.fit(X_train_tf, y_train_le)
  d_pred = d_model.predict(X_test_tf)
  results_df.loc[len(results_df.index)] = ['Decision Tree', size, accuracy_score(y_test.argmax(1), d_pred),
                                           f1_score(y_test.argmax(1), d_pred, average='micro'),
                                           f1_score(y_test.argmax(1), d_pred, average='macro'),
                                           fbeta_score(y_test.argmax(1), d_pred, beta=2, average='micro'),
                                           fbeta_score(y_test.argmax(1), d_pred, beta=2, average='macro'),
                                           log_loss(y_test, d_model.predict_proba(X_test_tf))]
  print("Decision Tree")

  # LOGISTIC REGRESSION
  l_model = LogisticRegression(C=10, penalty='l2')
  l_model.fit(X_train_tf, y_train_le)
  l_pred = l_model.predict(X_test_tf)
  results_df.loc[len(results_df.index)] = ['Logistic Regression', size, accuracy_score(y_test.argmax(1), l_pred),
                                           f1_score(y_test.argmax(1), l_pred, average='micro'),
                                           f1_score(y_test.argmax(1), l_pred, average='macro'),
                                           fbeta_score(y_test.argmax(1), l_pred, beta=2, average='micro'),
                                           fbeta_score(y_test.argmax(1), l_pred, beta=2, average='macro'),
                                           log_loss(y_test, l_model.predict_proba(X_test_tf))]
  print("Logistic Regression")

  # RANDOM FOREST
  r_model = RandomForestClassifier(max_depth=None, n_estimators=50)
  r_model.fit(X_train_tf, y_train_le)
  r_pred = r_model.predict(X_test_tf)
  results_df.loc[len(results_df.index)] = ['Random Forest', size, accuracy_score(y_test.argmax(1), r_pred),
                                           f1_score(y_test.argmax(1), r_pred, average='micro'),
                                           f1_score(y_test.argmax(1), r_pred, average='macro'),
                                           fbeta_score(y_test.argmax(1), r_pred, beta=2, average='micro'),
                                           fbeta_score(y_test.argmax(1), r_pred, beta=2, average='macro'),
                                           log_loss(y_test, r_model.predict_proba(X_test_tf))]

  print("Random Forest")

  # ROBERTA
  tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")
  token_lens = []
  for txt in X_train:
      tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
      token_lens.append(len(tokens))
  max_length=np.max(token_lens)
  MAX_LEN=max_length

  train_input_ids, train_attention_masks = tokenize_roberta(X_train, MAX_LEN)
  val_input_ids, val_attention_masks = tokenize_roberta(X_valid, MAX_LEN)
  test_input_ids, test_attention_masks = tokenize_roberta(X_test, MAX_LEN)
  roberta_model = TFRobertaModel.from_pretrained('roberta-base')

  model = create_model(roberta_model, MAX_LEN)
  # model.summary()
  # history =
  # model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid),
                        # epochs=7, batch_size=16)
  model.fit([train_input_ids,train_attention_masks], y_train, epochs=7, batch_size=16)

  result_roberta = model.predict([test_input_ids,test_attention_masks])
  y_pred_roberta =  np.zeros_like(result_roberta)
  y_pred_roberta[np.arange(len(y_pred_roberta)), result_roberta.argmax(1)] = 1

  # conf_matrix(y_test.argmax(1),y_pred_roberta.argmax(1),'RoBERTa Confusion Matrix')
  # print('\tClassification Report for RoBERTa:\n\n',classification_report(y_test,y_pred_roberta, target_names=['Not self identified', 'Self-identified']))
  results_df.loc[len(results_df.index)] = ['RoBERTa', size, accuracy_score(y_test.argmax(1),y_pred_roberta.argmax(1)),
                                           f1_score(y_test.argmax(1),y_pred_roberta.argmax(1), average='micro'),
                                           f1_score(y_test.argmax(1),y_pred_roberta.argmax(1), average='macro'),
                                           fbeta_score(y_test.argmax(1),y_pred_roberta.argmax(1), beta=2, average='micro'),
                                           fbeta_score(y_test.argmax(1),y_pred_roberta.argmax(1), beta=2, average='macro'),
                                           log_loss(y_test, model.predict([test_input_ids,test_attention_masks]))]
  print("RoBERTa")
  print("\n")

*******************************************
Currently running for size:  0.1
Naive Bayes
Decision Tree
Logistic Regression
Random Forest


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/7
22/22 [==============================] - 58s 466ms/step - loss: 0.6053 - categorical_accuracy: 0.7407
Epoch 2/7
22/22 [==============================] - 9s 415ms/step - loss: 0.5821 - categorical_accuracy: 0.7436
Epoch 3/7
22/22 [==============================] - 9s 411ms/step - loss: 0.5534 - categorical_accuracy: 0.7436
Epoch 4/7
22/22 [==============================] - 9s 405ms/step - loss: 0.4980 - categorical_accuracy: 0.7578
Epoch 5/7
22/22 [==============================] - 11s 527ms/step - loss: 0.3902 - categorical_accuracy: 0.7920
Epoch 6/7
22/22 [==============================] - 9s 393ms/step - loss: 0.2522 - categorical_accuracy: 0.9031
Epoch 7/7
10/10 [==============================] - 2s 237ms/step
RoBERTa


*******************************************
Currently running for size:  0.2
Naive Bayes
Decision Tree
Logistic Regression
Random Forest


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/7
44/44 [==============================] - 63s 453ms/step - loss: 0.5654 - categorical_accuracy: 0.7450
Epoch 2/7
44/44 [==============================] - 18s 403ms/step - loss: 0.5420 - categorical_accuracy: 0.7407
Epoch 3/7
44/44 [==============================] - 17s 395ms/step - loss: 0.4412 - categorical_accuracy: 0.7892
Epoch 4/7
44/44 [==============================] - 17s 390ms/step - loss: 0.3357 - categorical_accuracy: 0.8575
Epoch 5/7
44/44 [==============================] - 17s 395ms/step - loss: 0.1938 - categorical_accuracy: 0.9231
Epoch 6/7
44/44 [==============================] - 17s 396ms/step - loss: 0.1318 - categorical_accuracy: 0.9587
Epoch 7/7
10/10 [==============================] - 2s 229ms/step
RoBERTa


*******************************************
Currently running for size:  0.3
Naive Bayes
Decision Tree
Logistic Regression
Random Forest


...

Epoch 1/7
132/132 [==============================] - 99s 419ms/step - loss: 0.5629 - categorical_accuracy: 0.7356
Epoch 2/7
132/132 [==============================] - 52s 395ms/step - loss: 0.4109 - categorical_accuracy: 0.8282
Epoch 3/7
132/132 [==============================] - 52s 393ms/step - loss: 0.2861 - categorical_accuracy: 0.8889
Epoch 4/7
132/132 [==============================] - 52s 392ms/step - loss: 0.1977 - categorical_accuracy: 0.9250
Epoch 5/7
132/132 [==============================] - 52s 391ms/step - loss: 0.1284 - categorical_accuracy: 0.9549
Epoch 6/7
132/132 [==============================] - 52s 392ms/step - loss: 0.0912 - categorical_accuracy: 0.9696
Epoch 7/7
10/10 [==============================] - 2s 233ms/step
RoBERTa


*******************************************
Currently running for size:  0.7
Naive Bayes
Decision Tree
Logistic Regression
Random Forest


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/7
154/154 [==============================] - 107s 417ms/step - loss: 0.5360 - categorical_accuracy: 0.7556
Epoch 2/7
154/154 [==============================] - 61s 396ms/step - loss: 0.3632 - categorical_accuracy: 0.8406
Epoch 3/7
154/154 [==============================] - 61s 393ms/step - loss: 0.2612 - categorical_accuracy: 0.8979
Epoch 4/7
154/154 [==============================] - 60s 392ms/step - loss: 0.1855 - categorical_accuracy: 0.9325
Epoch 5/7
154/154 [==============================] - 60s 392ms/step - loss: 0.1281 - categorical_accuracy: 0.9528
Epoch 6/7
154/154 [==============================] - 60s 392ms/step - loss: 0.0839 - categorical_accuracy: 0.9711
Epoch 7/7
10/10 [==============================] - 2s 232ms/step
RoBERTa


*******************************************
Currently running for size:  0.8
Naive Bayes
Decision Tree
Logistic Regression
Random Forest


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/7
176/176 [==============================] - 116s 416ms/step - loss: 0.4900 - categorical_accuracy: 0.7851
Epoch 2/7
176/176 [==============================] - 69s 394ms/step - loss: 0.3387 - categorical_accuracy: 0.8630
Epoch 3/7
176/176 [==============================] - 69s 393ms/step - loss: 0.2426 - categorical_accuracy: 0.9064
Epoch 4/7
176/176 [==============================] - 69s 392ms/step - loss: 0.1812 - categorical_accuracy: 0.9356
Epoch 5/7
176/176 [==============================] - 69s 392ms/step - loss: 0.1224 - categorical_accuracy: 0.9559
Epoch 6/7
176/176 [==============================] - 69s 392ms/step - loss: 0.1038 - categorical_accuracy: 0.9641
Epoch 7/7
10/10 [==============================] - 2s 231ms/step
RoBERTa




In [ ]:
results_df.to_csv('result_dev.csv',index=False)